# Farina in Natura - estratégia

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime



In [3]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [45]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [46]:
transactions.head()

,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,46.0,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [47]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   item        25922 non-null  float64
 5   produto     25992 non-null  object 
 6   quantidade  25992 non-null  float64
 7   valor_unit  25992 non-null  object 
 8   desconto    25992 non-null  float64
 9   total_item  25992 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.0+ MB


# Limpeza e manipulação dos dados

In [ ]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

In [49]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [ ]:
# Delete the column "item"

del transactions["item"]

In [86]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   venda       25992 non-null  int64         
 1   data_hora   25992 non-null  datetime64[ns]
 2   cliente     25992 non-null  object        
 3   mesa        25992 non-null  object        
 4   produto     25992 non-null  object        
 5   quantidade  25992 non-null  int64         
 6   valor_unit  25992 non-null  float64       
 7   desconto    25992 non-null  float64       
 8   total_item  25992 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 2.0+ MB


,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
0,460,2022-02-01 08:13:00,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5
1,460,2022-02-01 08:13:00,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5
2,461,2022-02-01 08:14:00,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5
3,461,2022-02-01 08:14:00,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5
4,465,2022-02-01 09:01:00,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0


In [53]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [ ]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

In [56]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [ ]:
# Find Unique Values in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
transactions.produto.value_counts()

In [71]:
## Get Unique Count of using Series.unique()
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
transactions.produto.unique().size

212

In [79]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [76]:
# Convert "data" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [85]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [81]:
# change the decimal delimiter of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [83]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)